# Anthropic Prompt Caching 실험
아래의 노트북은 참조 노트북을 "짧은 한글 문서" 및 저렴한 Haiku3.0 으로 테스트한 내용 입니다. 

- 참조: [Prompt caching through the Anthropic API](https://github.com/anthropics/anthropic-cookbook/blob/main/misc/prompt_caching.ipynb)



# Prompt caching through the Anthropic API

Prompt caching allows you to store and reuse context within your prompt. This makes it more practical to include additional information in your prompt—such as detailed instructions and example responses—which help improve every response Claude generates.

In addition, by fully leveraging prompt caching within your prompt, you can reduce latency by >2x and costs up to 90%. This can generate significant savings when building solutions that involve repetitive tasks around detailed book_content.

In this cookbook, we will demonstrate how to use prompt caching in a single turn and across a multi-turn conversation. 


## Setup

First, let's set up our environment with the necessary imports and initializations:

In [24]:
%pip install anthropic bs4 --quiet

Note: you may need to restart the kernel to use updated packages.


In [25]:
import anthropic
import time
import requests
from bs4 import BeautifulSoup

client = anthropic.Anthropic(api_key="<Type API Key>")
# MODEL_NAME = "claude-3-5-sonnet-20241022"
MODEL_NAME = "claude-3-haiku-20240307"

Now let's fetch some text content to use in our examples. We'll use the text from Pride and Prejudice by Jane Austen which is around ~187,000 tokens long.

In [26]:
book_content = """
태양계는 우주에서 우리의 고향이라 할 수 있는 거대한 천체계입니다. 태양을 중심으로 8개의 행성과 그들의 위성, 소행성, 혜성 등 다양한 천체들이 끊임없이 운동하고 있는 하나의 거대한 시스템입니다. 약 46억 년 전 거대한 성간 물질 구름이 중력에 의해 수축하면서 형성되었으며, 현재까지도 끊임없이 진화하고 있습니다.
태양계의 중심인 태양은 전체 질량의 99.86%를 차지하는 거대한 플라즈마 덩어리입니다. 지름이 약 139만 킬로미터에 달하며, 수소와 헬륨의 핵융합 반응을 통해 엄청난 에너지를 방출합니다. 태양의 표면 온도는 약 5,500도이며, 중심부는 1,500만도에 달합니다. 이러한 태양의 강력한 중력이 행성들의 궤도를 유지하게 하는 핵심 요소입니다.
태양으로부터 가장 가까운 곳에 위치한 수성은 대기가 거의 없어 극심한 온도 변화를 겪습니다. 낮에는 430도까지 올라가고 밤에는 영하 180도까지 내려가는 극단적인 환경을 가지고 있습니다. 표면은 수많은 운석 충돌 흔적으로 가득하며, 위성은 보유하고 있지 않습니다.
금성은 크기와 질량이 지구와 비슷해 '지구의 쌍둥이'라고 불립니다. 하지만 두꺼운 이산화탄소 대기로 인한 극심한 온실효과로 표면 온도가 460도에 달해 태양계에서 가장 뜨거운 행성입니다. 특이하게도 다른 행성들과 반대 방향으로 자전하는 특징이 있습니다.
우리의 터전인 지구는 현재까지 발견된 생명체가 존재하는 유일한 행성입니다. 적당한 대기압과 온도, 액체 상태의 물, 자기장의 보호 등 생명체가 살아가기에 완벽한 조건을 갖추고 있습니다. 지구의 유일한 자연위성인 달은 조수현상을 일으키고 지구의 자전축을 안정화시키는 중요한 역할을 합니다.
화성은 인류의 다음 탐사 목표로 주목받고 있는 행성입니다. 붉은 색을 띠고 있어 '붉은 행성'으로도 불리며, 과거에 물이 흘렀던 흔적이 발견되어 생명체 존재 가능성에 대한 연구가 활발히 진행되고 있습니다. 두 개의 작은 위성인 포보스와 데이모스를 가지고 있습니다.
목성은 태양계 최대의 행성으로, 모든 행성 질량을 합한 것의 2.5배에 달합니다. 수소와 헬륨으로 이루어진 거대한 가스 행성이며, 대적점이라 불리는 거대한 폭풍이 수백 년간 지속되고 있습니다. 현재까지 79개의 위성이 발견되었으며, 희미한 고리도 가지고 있습니다.
토성은 아름다운 고리로 유명한 가스 행성입니다. 평균 밀도가 물보다 낮아 충분히 큰 물웅덩이가 있다면 떠오를 수 있을 정도입니다. 82개의 위성을 보유하고 있으며, 그중 타이탄은 두꺼운 대기층을 가진 가장 큰 위성입니다.
천왕성은 자전축이 공전면과 거의 수직을 이루고 있어 '누운 행성'으로도 불립니다. 메탄에 의해 청록색을 띠며, 27개의 알려진 위성과 희미한 고리를 가지고 있습니다. 계절의 변화가 매우 극단적으로 나타나는 특징이 있습니다.
해왕성은 태양계의 마지막 행성으로, 짙은 청색을 띠며 매우 강한 바람이 부는 것이 특징입니다. 14개의 위성을 가지고 있으며, 가장 큰 위성인 트리톤은 역방향으로 공전하는 특이한 특징을 보입니다.
태양계에는 이러한 행성들 외에도 다양한 천체들이 존재합니다. 화성과 목성 사이에 위치한 소행성대에는 수많은 작은 천체들이 분포하고 있으며, 해왕성 궤도 너머의 카이퍼 벨트에는 명왕성을 비롯한 왜소행성들이 있습니다. 가장 외곽에는 오르트 구름이 있어 많은 혜성의 기원이 되고 있습니다.
인류는 다양한 우주 탐사선을 통해 태양계를 연구하고 있습니다. 보이저호, 카시니호, 뉴호라이즌스 등의 탐사선들이 새로운 발견을 이어가고 있으며, 특히 화성 탐사와 생명체 탐색에 많은 노력을 기울이고 있습니다.
미래에는 화성 정착, 소행성 자원 채굴, 새로운 거주지 탐색 등이 주요 과제가 될 것으로 예상됩니다. 약 50억 년 후에는 태양이 적색거성으로 진화하면서 태양계의 구조가 크게 변화할 것으로 예측되고 있습니다.
이처럼 태양계는 끊임없이 변화하고 진화하는 역동적인 시스템이며, 인류에게 무한한 탐구의 대상이 되고 있습니다. 우주 기술의 발전과 함께 태양계에 대한 이해도 계속해서 깊어지고 있으며, 앞으로도 많은 새로운 발견이 이어질 것으로 기대됩니다.
"""

In [27]:
book_content

"\n태양계는 우주에서 우리의 고향이라 할 수 있는 거대한 천체계입니다. 태양을 중심으로 8개의 행성과 그들의 위성, 소행성, 혜성 등 다양한 천체들이 끊임없이 운동하고 있는 하나의 거대한 시스템입니다. 약 46억 년 전 거대한 성간 물질 구름이 중력에 의해 수축하면서 형성되었으며, 현재까지도 끊임없이 진화하고 있습니다.\n태양계의 중심인 태양은 전체 질량의 99.86%를 차지하는 거대한 플라즈마 덩어리입니다. 지름이 약 139만 킬로미터에 달하며, 수소와 헬륨의 핵융합 반응을 통해 엄청난 에너지를 방출합니다. 태양의 표면 온도는 약 5,500도이며, 중심부는 1,500만도에 달합니다. 이러한 태양의 강력한 중력이 행성들의 궤도를 유지하게 하는 핵심 요소입니다.\n태양으로부터 가장 가까운 곳에 위치한 수성은 대기가 거의 없어 극심한 온도 변화를 겪습니다. 낮에는 430도까지 올라가고 밤에는 영하 180도까지 내려가는 극단적인 환경을 가지고 있습니다. 표면은 수많은 운석 충돌 흔적으로 가득하며, 위성은 보유하고 있지 않습니다.\n금성은 크기와 질량이 지구와 비슷해 '지구의 쌍둥이'라고 불립니다. 하지만 두꺼운 이산화탄소 대기로 인한 극심한 온실효과로 표면 온도가 460도에 달해 태양계에서 가장 뜨거운 행성입니다. 특이하게도 다른 행성들과 반대 방향으로 자전하는 특징이 있습니다.\n우리의 터전인 지구는 현재까지 발견된 생명체가 존재하는 유일한 행성입니다. 적당한 대기압과 온도, 액체 상태의 물, 자기장의 보호 등 생명체가 살아가기에 완벽한 조건을 갖추고 있습니다. 지구의 유일한 자연위성인 달은 조수현상을 일으키고 지구의 자전축을 안정화시키는 중요한 역할을 합니다.\n화성은 인류의 다음 탐사 목표로 주목받고 있는 행성입니다. 붉은 색을 띠고 있어 '붉은 행성'으로도 불리며, 과거에 물이 흘렀던 흔적이 발견되어 생명체 존재 가능성에 대한 연구가 활발히 진행되고 있습니다. 두 개의 작은 위성인 포보스와 데이모스를 가지고 있습니다.\n목성은 태양계 최대의 행성으로, 모든

## Example 1: Single turn

Let's demonstrate prompt caching with a large document, comparing the performance and cost between cached and non-cached API calls.

### Part 1: Non-cached API Call

First, let's make a non-cached API call. This will load the prompt into the cache so that our subsequent cached API calls can benefit from the prompt caching.

We will ask for a short output string to keep the output response time low since the benefit of prompt caching applies only to the input processing time.

### 메세지 생성

In [35]:
def get_message():
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "<book>" + book_content + "</book>",
                    "cache_control": {"type": "ephemeral"}
                },
                {
                    "type": "text",
                    "text": "다른 행성들과 반대 방향으로 자전하는 행성은 뭐야? 답변은 단답식으로 말해줘. (예: 수성)"
                }
            ]
        }
    ]

    return messages

book_content_val = get_message()
book_content_val = book_content_val[0]['content'][0]['text']
print("book_content_val: ", len(book_content_val))




book_content_val:  2019


### API Call 을 하면서 캐시 Write 함

In [36]:
def make_non_cached_api_call():
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "<book>" + book_content + "</book>",
                    "cache_control": {"type": "ephemeral"}
                },
                {
                    "type": "text",
                    "text": "다른 행성들과 반대 방향으로 자전하는 행성은 뭐야? 답변은 단답식으로 말해줘. (예: 수성)"
                }
            ]
        }
    ]

    start_time = time.time()
    response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=300,
        messages=messages,
        extra_headers={"anthropic-beta": "prompt-caching-2024-07-31"}

    )
    end_time = time.time()

    return response, end_time - start_time

non_cached_response, non_cached_time = make_non_cached_api_call()

print(f"Non-cached API call time: {non_cached_time:.2f} seconds")
print(f"Non-cached API call input tokens: {non_cached_response.usage.input_tokens}")
print(f"Non-cached API call output tokens: {non_cached_response.usage.output_tokens}")
print(f"Non-cached API call cache_creation_input_tokens: {non_cached_response.usage.cache_creation_input_tokens}")
print(f"Non-cached API call cache_read_input_tokens: {non_cached_response.usage.cache_read_input_tokens}")


print("\nSummary (non-cached):")
print(non_cached_response.content)

Non-cached API call time: 0.48 seconds
Non-cached API call input tokens: 57
Non-cached API call output tokens: 10
Non-cached API call cache_creation_input_tokens: 2079
Non-cached API call cache_read_input_tokens: 0

Summary (non-cached):
[TextBlock(text='금성입니다.', type='text')]


In [37]:
non_cached_response

Message(id='msg_01EtsRGUr2Hj31PJ9pPJkny3', content=[TextBlock(text='금성입니다.', type='text')], model='claude-3-haiku-20240307', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=Usage(input_tokens=57, output_tokens=10, cache_creation_input_tokens=2079, cache_read_input_tokens=0))

### Part 2: Cached API Call

Now, let's make a cached API call. I'll add in the "cache_control": {"type": "ephemeral"} attribute to the content object and add the "prompt-caching-2024-07-31" beta header to the request. This will enable prompt caching for this API call.

To keep the output latency constant, we will ask Claude the same question as before. Note that this question is not part of the cached content.

In [38]:
def make_cached_api_call():
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "<book>" + book_content + "</book>",
                    "cache_control": {"type": "ephemeral"}
                },
                {
                    "type": "text",
                    "text": "다른 행성들과 반대 방향으로 자전하는 행성은 뭐야? 답변은 단답식으로 말해줘. (예: 수성)"
                }
            ]
        }
    ]

    start_time = time.time()
    response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=300,
        messages=messages,
        extra_headers={"anthropic-beta": "prompt-caching-2024-07-31"}
    )
    end_time = time.time()

    return response, end_time - start_time

cached_response, cached_time = make_cached_api_call()

print(f"Cached API call time: {cached_time:.2f} seconds")
print(f"Cached API call input tokens: {cached_response.usage.input_tokens}")
print(f"Cached API call output tokens: {cached_response.usage.output_tokens}")
print(f"Non-cached API call cache_creation_input_tokens: {cached_response.usage.cache_creation_input_tokens}")
print(f"Non-cached API call cache_read_input_tokens: {cached_response.usage.cache_read_input_tokens}")

print("\nSummary (cached):")
print(cached_response.content)

Cached API call time: 0.54 seconds
Cached API call input tokens: 57
Cached API call output tokens: 6
Non-cached API call cache_creation_input_tokens: 0
Non-cached API call cache_read_input_tokens: 2079

Summary (cached):
[TextBlock(text='금성', type='text')]


In [39]:
cached_response

Message(id='msg_01JG6yoCafqJpV8DnapHDWjy', content=[TextBlock(text='금성', type='text')], model='claude-3-haiku-20240307', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=Usage(input_tokens=57, output_tokens=6, cache_creation_input_tokens=0, cache_read_input_tokens=2079))

As you can see, the cached API call only took 3.64 seconds total compared to 21.44 seconds for the non-cached API call. This is a significant improvement in overall latency due to caching.

## Example 2: Multi-turn Conversation with Incremental Caching

Now, let's look at a multi-turn conversation where we add cache breakpoints as the conversation progresses.

In [40]:
class ConversationHistory:
    def __init__(self):
        # Initialize an empty list to store conversation turns
        self.turns = []

    def add_turn_assistant(self, content):
        # Add an assistant's turn to the conversation history
        self.turns.append({
            "role": "assistant",
            "content": [
                {
                    "type": "text",
                    "text": content
                }
            ]
        })

    def add_turn_user(self, content):
        # Add a user's turn to the conversation history
        self.turns.append({
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": content
                }
            ]
        })

    def get_turns(self):
        # Retrieve conversation turns with specific formatting
        result = []
        user_turns_processed = 0
        # Iterate through turns in reverse order
        for turn in reversed(self.turns):
            if turn["role"] == "user" and user_turns_processed < 2:
                # Add the last two user turns with ephemeral cache control
                result.append({
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": turn["content"][0]["text"],
                            "cache_control": {"type": "ephemeral"}
                        }
                    ]
                })
                user_turns_processed += 1
            else:
                # Add other turns as they are
                result.append(turn)
        # Return the turns in the original order
        return list(reversed(result))

# Initialize the conversation history
conversation_history = ConversationHistory()

# System message containing the book content
# Note: 'book_content' should be defined elsewhere in the code
system_message = f"<file_contents> {book_content} </file_contents>"

# Predefined questions for our simulation
questions = [
    "다른 행성들과 반대 방향으로 자전하는 행성은 뭐야? 답변은 단답식으로 말해줘. (예: 수성)",
    "붉은 색을 띠고 있어 붉은 행성 으로도 불리며, 과거에 물이 흘렀던 흔적이 발견되어 생명체 존재 가능성이 있는 행성은 뭐야? 답변은 단답식으로 말해줘. (예: 수성)",
    "82개의 위성을 보유하고 있으며, 그중 타이탄은 두꺼운 대기층을 가진 가장 큰 위성을 가진 행성은 뭐야? 답변은 단답식으로 말해줘. (예: 수성)",
    "짙은 청색을 띠며 매우 강한 바람이 부는 것이 특징인 행성은 뭐야? 답변은 단답식으로 말해줘. (예: 수성)"
]

def simulate_conversation():
    for i, question in enumerate(questions, 1):
        print(f"\nTurn {i}:")
        print(f"User: {question}")
        
        # Add user input to conversation history
        conversation_history.add_turn_user(question)
        messages=conversation_history.get_turns()
        print("## messages: \n", messages)

        # Record the start time for performance measurement
        start_time = time.time()

        # Make an API call to the assistant
        response = client.messages.create(
            model=MODEL_NAME,
            extra_headers={
              "anthropic-beta": "prompt-caching-2024-07-31"
            },
            max_tokens=300,
            system=[
                {"type": "text", "text": system_message, "cache_control": {"type": "ephemeral"}},
            ],
            messages=messages,
        )

        # Record the end time
        end_time = time.time()

        # Extract the assistant's reply
        assistant_reply = response.content[0].text
        print(f"Assistant: {assistant_reply}")

        # Print token usage information
        input_tokens = response.usage.input_tokens
        output_tokens = response.usage.output_tokens
        input_tokens_cache_read = getattr(response.usage, 'cache_read_input_tokens', '---')
        input_tokens_cache_create = getattr(response.usage, 'cache_creation_input_tokens', '---')
        print(f"User input tokens: {input_tokens}")
        print(f"Output tokens: {output_tokens}")
        print(f"Input tokens (cache read): {input_tokens_cache_read}")
        print(f"Input tokens (cache write): {input_tokens_cache_create}")

        # Calculate and print the elapsed time
        elapsed_time = end_time - start_time

        # Calculate the percentage of input prompt cached
        total_input_tokens = input_tokens + (int(input_tokens_cache_read) if input_tokens_cache_read != '---' else 0)
        percentage_cached = (int(input_tokens_cache_read) / total_input_tokens * 100 if input_tokens_cache_read != '---' and total_input_tokens > 0 else 0)

        print(f"{percentage_cached:.1f}% of input prompt cached ({total_input_tokens} tokens)")
        print(f"Time taken: {elapsed_time:.2f} seconds")

        # Add assistant's reply to conversation history
        conversation_history.add_turn_assistant(assistant_reply)

# Run the simulated conversation
simulate_conversation()


Turn 1:
User: 다른 행성들과 반대 방향으로 자전하는 행성은 뭐야? 답변은 단답식으로 말해줘. (예: 수성)
## messages: 
 [{'role': 'user', 'content': [{'type': 'text', 'text': '다른 행성들과 반대 방향으로 자전하는 행성은 뭐야? 답변은 단답식으로 말해줘. (예: 수성)', 'cache_control': {'type': 'ephemeral'}}]}]
Assistant: 금성
User input tokens: 4
Output tokens: 6
Input tokens (cache read): 0
Input tokens (cache write): 2139
0.0% of input prompt cached (4 tokens)
Time taken: 0.46 seconds

Turn 2:
User: 붉은 색을 띠고 있어 붉은 행성 으로도 불리며, 과거에 물이 흘렀던 흔적이 발견되어 생명체 존재 가능성이 있는 행성은 뭐야? 답변은 단답식으로 말해줘. (예: 수성)
## messages: 
 [{'role': 'user', 'content': [{'type': 'text', 'text': '다른 행성들과 반대 방향으로 자전하는 행성은 뭐야? 답변은 단답식으로 말해줘. (예: 수성)', 'cache_control': {'type': 'ephemeral'}}]}, {'role': 'assistant', 'content': [{'type': 'text', 'text': '금성'}]}, {'role': 'user', 'content': [{'type': 'text', 'text': '붉은 색을 띠고 있어 붉은 행성 으로도 불리며, 과거에 물이 흘렀던 흔적이 발견되어 생명체 존재 가능성이 있는 행성은 뭐야? 답변은 단답식으로 말해줘. (예: 수성)', 'cache_control': {'type': 'ephemeral'}}]}]
Assistant: 화성
User input tokens: 4
Output tokens: 

As you can see in this example, response times decreased from nearly 24 seconds to just 7-11 seconds after the initial cache setup, while maintaining the same level of quality across the answers. Most of this remaining latency is due to the time it takes to generate the response, which is not affected by prompt caching.

And since nearly 100% of input tokens were cached in subsequent turns as we kept adjusting the cache breakpoints, we were able to read the next user message nearly instantly.